## **`System Information AI Agent`**

List of columns that are extract here 
- Market Segment	
- System Type (General)	
- System Type (Specific)	
- System Name (General)	
- System Name (Specific)	
- System Piloting

In [19]:
import os
import getpass
import json
import pandas as pd
from openai import OpenAI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from pydantic import BaseModel, Field
from langgraph.graph import MessagesState, StateGraph, END, START
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI

# ------------------------------------------------------------------
# 1. CONFIGURATION: THE "RULE BOOK" (SCALABLE LOGIC)
# ------------------------------------------------------------------
RULE_BOOK = {
    "defensive_countermeasures": {
        "triggers": ["flare", "chaff", "countermeasure", "decoy", "mju-", "ale-"],
        "guidance": """
            - CONTEXT: Items like Flares, Chaff, and Decoys are used for PROTECTION, not attack.
            - MAPPING: 
                * Market Segment: 'C4ISR Systems'
                * System Type (General): 'Defensive Systems'
                * System Type (Specific): 'Defensive Aid Suite'
        """
    },
    "radars_and_sensors": {
        "triggers": ["radar", "sonar", "sensor", "an/apy", "an/tpy"],
        "guidance": """
            - CONTEXT: Detection equipment.
            - MAPPING:
                * Market Segment: 'C4ISR Systems'
                * System Type (General): 'Sensors'
        """
    },
    "ammunition": {
        "triggers": ["cartridge", "round", "projectile", " 5.56", " 7.62", "ammo"],
        "guidance": """
            - CONTEXT: Lethal rounds or kinetic energy penetrators.
            - MAPPING:
                * Market Segment: 'Weapon Systems'
                * System Type (General): 'Ammunition' (Unless it's a Flare/Decoy -> see Defensive rules).
        """
    }
}

# ------------------------------------------------------------------
# 2. SETUP (API & FILES)
# ------------------------------------------------------------------

if "LLMFOUNDRY_TOKEN" not in os.environ:
    os.environ["LLMFOUNDRY_TOKEN"] = getpass.getpass("Enter the LLM Foundry API Key: ")

client = OpenAI(
    api_key=f'{os.environ.get("LLMFOUNDRY_TOKEN")}:my-test-project',
    base_url="https://llmfoundry.straive.com/openai/v1/",
)

SAMPLE_FILE = r"C:\Users\mukeshkr\Downloads\AN_Defense_Data.xlsx"
TAXONOMY_FILE = r"C:\Users\mukeshkr\Downloads\taxonomy.json"

def load_taxonomy(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.dumps(json.load(f), separators=(',', ':'))
    except FileNotFoundError:
        return "{}"

taxonomy_str = load_taxonomy(TAXONOMY_FILE)

# ------------------------------------------------------------------
# 3. RETRIEVAL LOGIC (Unchanged)
# ------------------------------------------------------------------

try:
    df_examples = pd.read_excel(SAMPLE_FILE, header=1)
    df_examples = df_examples.fillna("Not Applicable")
    vectorizer = TfidfVectorizer(stop_words='english')
    example_vectors = vectorizer.fit_transform(df_examples['Description of Contract'])
except FileNotFoundError:
    df_examples = pd.DataFrame()
    vectorizer = None
    example_vectors = None

def get_similar_example(new_text):
    if vectorizer is None or df_examples.empty:
        return None
    new_vec = vectorizer.transform([new_text])
    similarities = cosine_similarity(new_vec, example_vectors).flatten()
    best_idx = similarities.argmax()
    if similarities[best_idx] > 0.1:
        row = df_examples.iloc[best_idx]
        return {
            "text": row['Description of Contract'],
            "classification": {
                "Market Segment": row['Market Segment'],
                "System Type (General)": row['System Type (General)'],
                "System Type (Specific)": row['System Type (Specific)'],
                "System Name (General)": row['System Name (General)'],
                "System Name (Specific)": row['System Name (Specific)'],
                "System Piloting": row.get('System Piloting', "Derived from logic")
            }
        }
    return None

# ------------------------------------------------------------------
# 4. THE SMART TOOL
# ------------------------------------------------------------------

class ContractInput(BaseModel):
    paragraph: str = Field(description="The defense contract text description to classify.")

@tool("defense_contract_classifier", args_schema=ContractInput)
def defense_contract_classifier(paragraph: str):
    """
    Classifies defense contracts using a dynamic Rule Book and historical examples.
    """
    
    # 1. Retrieve Historical Example (RAG)
    similar_case = get_similar_example(paragraph)

    # 2. Dynamic Rule Injection (The Scalable Part)
    # We check if any triggers in our RULE_BOOK appear in the text.
    active_rules = []
    lower_text = paragraph.lower()
    
    for rule_name, rule_data in RULE_BOOK.items():
        # Check if any trigger word exists in the input text
        if any(trigger in lower_text for trigger in rule_data["triggers"]):
            active_rules.append(f"--- RULE FOR {rule_name.upper()} ---\n{rule_data['guidance']}")
            
    # Join all active rules into a single string
    dynamic_hints = "\n".join(active_rules)

    # 3. Construct the Prompt
    system_instruction = f"""
    You are a Defense Contract Analyst.
    Classify the contract accurately. 

    REFERENCE TAXONOMY:
    {taxonomy_str}
    
    SPECIALIZED RULE BOOK (Apply these overrides if relevant):
    {dynamic_hints if dynamic_hints else "No special overrides detected. Use general logic."}
    
    NAMING CONVENTION (Apply Strictly):
    1. If a specific Alphanumeric Designator exists (e.g., 'MJU-61A/B', 'F-35', 'AN/SPY-1'):
       - Use it for BOTH 'System Name (General)' and 'System Name (Specific)'.
       - DO NOT use generic words like "Flare", "Radar", or "Jet" in the name fields if a designator is available.
    2. If NO designator exists, use the common platform name (e.g. 'Abrams').
    """

    user_message = f"Input Text: {paragraph}\n\n"

    if similar_case:
        user_message += f"""
        PAST EXAMPLE (For reference only - The RULE BOOK above takes priority):
        [Text]: {similar_case['text'][:300]}...
        [Class]: {json.dumps(similar_case['classification'])}
        """

    user_message += """
    --------------------------------------------------------
    Your Task:
    1. Analyze the 'Function' of the system (Protect, Attack, Transport, Sense).
    2. Check the 'Specialized Rule Book' above. Does the item fit a defined rule?
    3. Extract the names using the Naming Convention.

    Return JSON:
    {
      "reasoning": "Step-by-step logic explaining why you chose this path...",
      "classification": {
          "Market Segment": "...",
          "System Type (General)": "...",
          "System Type (Specific)": "...",
          "System Name (General)": "...",
          "System Name (Specific)": "...",
          "System Piloting": "..."
      }
    }
    """

    # 4. Call LLM
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_instruction},
            {"role": "user", "content": user_message}
        ],
        temperature=0,
        response_format={"type": "json_object"}
    )

    result = json.loads(completion.choices[0].message.content)
    return result.get("classification", result)

# ------------------------------------------------------------------
# 5. LANGGRAPH (Unchanged)
# ------------------------------------------------------------------

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=f'{os.environ.get("LLMFOUNDRY_TOKEN")}:my-test-project',
    base_url="https://llmfoundry.straive.com/openai/v1/",
    temperature=0
)

tools = [defense_contract_classifier]
llm_with_tools = llm.bind_tools(tools)

def call_model(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

tool_node = ToolNode(tools)

def should_continue(state: MessagesState):
    if state["messages"][-1].tool_calls:
        return "tools"
    return END

workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

app = workflow.compile()

if __name__ == "__main__":
    # Test text
    test_text = "General Dynamics NASSCO – San Diego, San Diego, California, is awarded a $98,749,123, firm-fixed-price contract for the execution of the USS Pinckney (DDG 91) FY22 depot modernization period. This availability will include a combination of maintenance, modernization and repair of the USS Pinckney. This contract includes options which, if exercised, would bring the cumulative value of this contract to $121,109,530. Work will be performed in San Diego, California, and is expected to be completed by July 2023. Fiscal 2021 other procurement (Navy) funds in the amount of $93,500,245 (94.7%); and fiscal 2021 operation and maintenance (Navy) funds in the amount of $5,248,878 (5.3%) will be obligated at time of award, of which, funding in the amount of $5,248,878 will expire at the end of the current fiscal year. This contract was competitively procured using full and open competition via the beta.sam.gov website with three offers were received in response to Solicitation No. N00024-21-R-4478.  The Naval Sea Systems Command, Washington, D.C., is the contracting activity (N00024-21-C-4478)."
    
    inputs = {"messages": [HumanMessage(content=f"Classify this contract: {test_text}")]}
    for chunk in app.stream(inputs, stream_mode="values"):
        msg = chunk["messages"][-1]
        msg.pretty_print()

================================ Human Message =================================

Classify this contract: General Dynamics NASSCO – San Diego, San Diego, California, is awarded a $98,749,123, firm-fixed-price contract for the execution of the USS Pinckney (DDG 91) FY22 depot modernization period. This availability will include a combination of maintenance, modernization and repair of the USS Pinckney. This contract includes options which, if exercised, would bring the cumulative value of this contract to $121,109,530. Work will be performed in San Diego, California, and is expected to be completed by July 2023. Fiscal 2021 other procurement (Navy) funds in the amount of $93,500,245 (94.7%); and fiscal 2021 operation and maintenance (Navy) funds in the amount of $5,248,878 (5.3%) will be obligated at time of award, of which, funding in the amount of $5,248,878 will expire at the end of the current fiscal year. This contract was competitively procured using full and open competition via 

## `**Contract Information**`

- Supplier Name	
- Program Type	
- Expected MRO Contract Duration (Months)	
- Quantity	
- Value Certainty	 
- Value (Million) 	
- Currency	
- Value (USD$ Million)	
- "Value Note(If Any)"	
- G2G/B2G	
- Signing Month	
- Signing Year


In [20]:
import os
import json
import difflib
import pandas as pd
from dateutil import parser
from dateutil.relativedelta import relativedelta
import getpass

# LangChain / LangGraph Imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from pydantic import BaseModel, Field
from langgraph.graph import MessagesState, StateGraph, END, START
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
from openai import OpenAI

In [21]:
# Setup API Key
if "LLMFOUNDRY_TOKEN" not in os.environ:
    os.environ["LLMFOUNDRY_TOKEN"] = getpass.getpass("Enter the LLM Foundry API Key: ")

client = OpenAI(
    api_key=f'{os.environ.get("LLMFOUNDRY_TOKEN")}:my-test-project',
    base_url="https://llmfoundry.straive.com/openai/v1/",
)

In [22]:
# --- LOAD SUPPLIERS FROM JSON FILE ---
def load_supplier_list(filename="suppliers.json"):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        # Fallback list for testing if file is missing
        print(f"⚠️ Warning: {filename} not found. Using fallback list.")
        return ["Dell Inc", "Boeing", "Lockheed Martin", "Raytheon Technologies"]

SUPPLIER_LIST = load_supplier_list()

In [24]:
len(SUPPLIER_LIST)

1724

In [25]:
def get_best_supplier_match(extracted_name):
    """
    Matches the raw extracted name against the loaded SUPPLIER_LIST.
    Running this locally costs 0 tokens.
    """
    if not extracted_name or extracted_name.lower() in ["unknown", "not applicable", "n/a"]:
        return "Unknown"

    clean_name = extracted_name.strip()
    
    # 1. Exact Match (Case Insensitive)
    supplier_map = {s.lower(): s for s in SUPPLIER_LIST}
    if clean_name.lower() in supplier_map:
        return supplier_map[clean_name.lower()]

    # 2. Fuzzy Match (Finds 'Dell Marketing' -> 'Dell Inc')
    # cutoff=0.6 means 60% similarity required.
    matches = difflib.get_close_matches(clean_name, SUPPLIER_LIST, n=1, cutoff=0.6)
    
    if matches:
        return matches[0]

    # 3. No Match? Return raw name (or "Unknown" if you prefer strictness)
    return clean_name

def calculate_mro_months(start_date_str, end_date_text):
    try:
        if not start_date_str or not end_date_text or end_date_text.lower() in ["unknown", "n/a"]:
            return "Not Applicable"
        start_date = pd.to_datetime(start_date_str, dayfirst=True)
        end_date = parser.parse(end_date_text, fuzzy=True)
        diff = relativedelta(end_date, start_date)
        return str(max(0, int((diff.years * 12) + diff.months)))
    except Exception:
        return "Not Applicable"

In [26]:
class ContractInfoInput(BaseModel):
    paragraph: str = Field(description="The defense contract text description.")
    contract_date: str = Field(description="The date the contract was signed (e.g., '2023-10-15').")

@tool("contract_info_extractor", args_schema=ContractInfoInput)
def contract_info_extractor(paragraph: str, contract_date: str):
    """
    Extracts financial and program details. 
    Matches Supplier Name locally using the JSON list.
    """

    # --- LIGHTWEIGHT PROMPT (No List Included) ---
    system_instruction = """
    You are a Defense Contract Financial Analyst.
    Extract data strictly following these rules:
    
    1. **Supplier Name**: Extract the exact company name string found in the text (e.g., "Dell Marketing L.P.").
    2. **Program Type**: Procurement, Training (Services), MRO/Support, RDT&E, or Other Service.
    3. **Value Certainty**: Confirmed (specific amount) vs Estimated (ceiling/potential).
    4. **G2G/B2G**: G2G if FMS mentioned; else B2G.
    
    Return JSON only.
    """

    user_prompt = f"""
    Analyze this text: "{paragraph}"
    Contract Date: {contract_date}

    Return JSON:
    {{
        "raw_supplier_text": "Extract exact company name text",
        "program_type": "Procurement/Training/MRO/Support/RDT&E/Upgrade/Other Service",
        "completion_date_text": "Extracted date string or null",
        "quantity": "Number or 'Not Applicable'",
        "value_certainty": "Confirmed/Estimated",
        "value_million": "Number (e.g. 2493.000)",
        "currency": "Code (e.g. USD$)",
        "g2g_b2g": "G2G/B2G",
        "value_note": "Note if split/ambiguous or 'Not Applicable'"
    }}
    """

    # --- CALL LLM ---
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        raw = json.loads(completion.choices[0].message.content)
    except Exception as e:
        return {"Error": f"LLM Failure: {str(e)}"}

    # --- LOCAL PYTHON MATCHING ---
    matched_supplier = get_best_supplier_match(raw.get("raw_supplier_text"))

    # MRO Calculation
    mro_months = "Not Applicable"
    if raw.get("program_type") == "MRO/Support":
        mro_months = calculate_mro_months(contract_date, raw.get("completion_date_text"))
    
    # Date Formatting
    try:
        sign_dt = pd.to_datetime(contract_date, dayfirst=True)
        sign_month = sign_dt.strftime("%B")
        sign_year = str(sign_dt.year)
    except:
        sign_month, sign_year = "Unknown", "Unknown"

    return {
        "Supplier Name": matched_supplier,
        "Program Type": raw.get("program_type", "Unknown"),
        "Expected MRO Contract Duration (Months)": mro_months,
        "Quantity": raw.get("quantity", "Not Applicable"),
        "Value Certainty": raw.get("value_certainty", "Confirmed"),
        "Value (Million)": raw.get("value_million", "0.000"),
        "Currency": raw.get("currency", "USD$"),
        "Value (USD$ Million)": raw.get("value_million", "0.000"),
        "Value Note (If Any)": raw.get("value_note", "Not Applicable"),
        "G2G/B2G": raw.get("g2g_b2g", "B2G"),
        "Signing Month": sign_month,
        "Signing Year": sign_year
    }

In [27]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=f'{os.environ.get("LLMFOUNDRY_TOKEN")}:my-test-project',
    base_url="https://llmfoundry.straive.com/openai/v1/",
    temperature=0
)

tools = [contract_info_extractor]
llm_with_tools = llm.bind_tools(tools)

def call_model(state: MessagesState):
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

tool_node = ToolNode(tools)

def should_continue(state: MessagesState):
    messages = state["messages"]
    if messages[-1].tool_calls:
        return "tools"
    return END

workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

app = workflow.compile()

In [28]:
if __name__ == "__main__":
    test_text = "General Dynamics NASSCO – San Diego, San Diego, California, is awarded a $98,749,123, firm-fixed-price contract for the execution of the USS Pinckney (DDG 91) FY22 depot modernization period. This availability will include a combination of maintenance, modernization and repair of the USS Pinckney. This contract includes options which, if exercised, would bring the cumulative value of this contract to $121,109,530. Work will be performed in San Diego, California, and is expected to be completed by July 2023. Fiscal 2021 other procurement (Navy) funds in the amount of $93,500,245 (94.7%); and fiscal 2021 operation and maintenance (Navy) funds in the amount of $5,248,878 (5.3%) will be obligated at time of award, of which, funding in the amount of $5,248,878 will expire at the end of the current fiscal year. This contract was competitively procured using full and open competition via the beta.sam.gov website with three offers were received in response to Solicitation No. N00024-21-R-4478.  The Naval Sea Systems Command, Washington, D.C., is the contracting activity (N00024-21-C-4478)."
    
    user_query = f"""
    Extract info.
    Text: {test_text}
    Date: 2021-04-15
    """

    print("--- Running Agent ---")
    inputs = {"messages": [HumanMessage(content=user_query)]}
    
    for chunk in app.stream(inputs, stream_mode="values"):
        message = chunk["messages"][-1]
        message.pretty_print()

--- Running Agent ---
================================ Human Message =================================


    Extract info.
    Text: General Dynamics NASSCO – San Diego, San Diego, California, is awarded a $98,749,123, firm-fixed-price contract for the execution of the USS Pinckney (DDG 91) FY22 depot modernization period. This availability will include a combination of maintenance, modernization and repair of the USS Pinckney. This contract includes options which, if exercised, would bring the cumulative value of this contract to $121,109,530. Work will be performed in San Diego, California, and is expected to be completed by July 2023. Fiscal 2021 other procurement (Navy) funds in the amount of $93,500,245 (94.7%); and fiscal 2021 operation and maintenance (Navy) funds in the amount of $5,248,878 (5.3%) will be obligated at time of award, of which, funding in the amount of $5,248,878 will expire at the end of the current fiscal year. This contract was competitively procured using ful

C:\Users\mukeshkr\AppData\Local\Temp\ipykernel_11572\4045033176.py:30: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  start_date = pd.to_datetime(start_date_str, dayfirst=True)
C:\Users\mukeshkr\AppData\Local\Temp\ipykernel_11572\3029917481.py:68: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  sign_dt = pd.to_datetime(contract_date, dayfirst=True)


================================= Tool Message =================================
Name: contract_info_extractor

{"Supplier Name": "General Dynamics", "Program Type": "MRO/Support", "Expected MRO Contract Duration (Months)": "27", "Quantity": "Not Applicable", "Value Certainty": "Confirmed", "Value (Million)": "98.749", "Currency": "USD$", "Value (USD$ Million)": "98.749", "Value Note (If Any)": "Not Applicable", "G2G/B2G": "B2G", "Signing Month": "April", "Signing Year": "2021"}
================================== Ai Message ==================================

Here is the extracted information from the contract:

- **Supplier Name**: General Dynamics
- **Program Type**: MRO/Support
- **Expected MRO Contract Duration (Months)**: 27
- **Quantity**: Not Applicable
- **Value Certainty**: Confirmed
- **Value (Million)**: 98.749
- **Currency**: USD$
- **Value (USD$ Million)**: 98.749
- **Value Note (If Any)**: Not Applicable
- **G2G/B2G**: B2G
- **Signing Month**: April
- **Signing Year**: 20

# **Customer/Supplier Geography**

- Customer Region	
- Customer Country	
- Customer Operator	
- Supplier Region	
- Supplier Country	
- Domestic Content


In [29]:
import os
import json
import getpass

# LangChain / LangGraph Imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from pydantic import BaseModel, Field
from langgraph.graph import MessagesState, StateGraph, END, START
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
from openai import OpenAI

In [30]:
if "LLMFOUNDRY_TOKEN" not in os.environ:
    os.environ["LLMFOUNDRY_TOKEN"] = getpass.getpass("Enter the LLM Foundry API Key: ")

client = OpenAI(
    api_key=f'{os.environ.get("LLMFOUNDRY_TOKEN")}:my-test-project',
    base_url="https://llmfoundry.straive.com/openai/v1/",
)

In [44]:
GEOGRAPHY_MAPPING = {
    "Sub-Saharan Africa": [
        "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cameroon", "Cape Verde",
        "Central African Republic", "Chad", "Comoros", "Congo", "Djibouti", "Equatorial Guinea", 
        "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea", "Ivory Coast", 
        "Kenya", "Lesotho", "Liberia", "Madagascar", "Malawi", "Mali", "Mauritania", "Mauritius", 
        "Mozambique", "Namibia", "Niger", "Nigeria", "Rwanda", "Sao Tome", "Senegal", "Seychelles", 
        "Sierra Leone", "Somalia", "South Africa", "South Sudan", "Sudan", "Tanzania", "Togo", 
        "Uganda", "Zambia", "Zimbabwe"
    ],
    "Asia-Pacific": [
        "Australia", "Bangladesh", "Bhutan", "Brunei", "Cambodia", "China", "Fiji", "India", 
        "Indonesia", "Japan", "Kiribati", "Laos", "Malaysia", "Maldives", "Marshall Islands", 
        "Micronesia", "Mongolia", "Myanmar", "Nauru", "Nepal", "New Zealand", "North Korea", 
        "Pakistan", "Palau", "Papua New Guinea", "Philippines", "Samoa", "Singapore", 
        "Solomon Islands", "South Korea", "Sri Lanka", "Taiwan", "Thailand", "Timor-Leste", 
        "Tonga", "Tuvalu", "Vanuatu", "Vietnam"
    ],
    "Europe": [
        "Albania", "Andorra", "Austria", "Belarus", "Belgium", "Bosnia", "Bulgaria", "Croatia", 
        "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Germany", 
        "Greece", "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", "Liechtenstein", 
        "Lithuania", "Luxembourg", "Malta", "Moldova", "Monaco", "Montenegro", "Netherlands", 
        "North Macedonia", "Norway", "Poland", "Portugal", "Romania", "San Marino", "Serbia", 
        "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland", "Ukraine", "United Kingdom", 
        "UK", "Great Britain", "England", "Vatican City"
    ],
    "Latin America": [
        "Antigua", "Argentina", "Bahamas", "Barbados", "Belize", "Bolivia", "Brazil", "Chile", 
        "Colombia", "Costa Rica", "Cuba", "Dominica", "Dominican Republic", "Ecuador", 
        "El Salvador", "Grenada", "Guatemala", "Guyana", "Haiti", "Honduras", "Jamaica", 
        "Mexico", "Nicaragua", "Panama", "Paraguay", "Peru", "St. Kitts", "St. Lucia", 
        "St. Vincent", "Suriname", "Trinidad", "Uruguay", "Venezuela"
    ],
    "Middle East and North Africa": [
        "Algeria", "Bahrain", "Egypt", "Iran", "Iraq", "Israel", "Jordan", "Kuwait", "Lebanon", 
        "Libya", "Morocco", "Oman", "Palestinian Territories", "Qatar", "Saudi Arabia", "Syria", 
        "Tunisia", "Turkey", "UAE", "United Arab Emirates", "Yemen"
    ],
    "North America": [
        "USA", "United States", "United States of America", "US", "U.S.", "Canada"
    ],
    "Russia & CIS": [
        "Russia", "Armenia", "Azerbaijan", "Kazakhstan", "Kyrgyzstan", "Tajikistan", 
        "Turkmenistan", "Uzbekistan", "Georgia"
    ],
    "International Organisations": [
        "NATO", "EU", "European Union", "IFU", "United Nations", "UN", "NSPA", "OCCAR"
    ]
}

In [45]:
def get_region_for_country(country_name):
    """
    Robust lookup that handles whitespace, casing, and aliases.
    """
    if not country_name or country_name.lower() in ["unknown", "not applicable", "n/a"]:
        return "Unknown"
    
    # 1. CLEAN THE INPUT (Strip whitespace & lowercase)
    clean_input = country_name.strip().lower()
    
    # 2. CHECK ALIASES DIRECTLY (Fast Path)
    if clean_input in ["us", "usa", "u.s.", "united states"]:
        return "North America"
    if clean_input in ["uk", "u.k.", "britain"]:
        return "Europe"

    # 3. SEARCH MAPPING
    for region, countries in GEOGRAPHY_MAPPING.items():
        # Create a lowercase set for this region for fast checking
        # (In production, do this once outside the function)
        country_set = {c.lower() for c in countries}
        
        if clean_input in country_set:
            return region
            
    return "Unknown"

In [46]:
def determine_domestic_content(cust_country, supp_country, special_flags):
    """Calculates Domestic Content logic."""
    if cust_country == "Unknown" or supp_country == "Unknown":
        return "Unknown"

    c_norm = cust_country.lower().replace("united states", "usa")
    s_norm = supp_country.lower().replace("united states", "usa")

    if special_flags.get("is_license_production"): return "License Production"
    if special_flags.get("is_local_assembly"): return "Local Assembly"

    return "Indigenous" if c_norm == s_norm else "Imported"

In [47]:
class GeographyInput(BaseModel):
    paragraph: str = Field(description="The defense contract text.")

@tool("geography_extractor", args_schema=GeographyInput)
def geography_extractor(paragraph: str):
    """
    Extracts geography info with a reasoning step to correctly identify the Customer Operator.
    """
    
    system_instruction = """
    You are a Senior Defense Geography Analyst.
    
    ### CRITICAL TASK: DETERMINE 'CUSTOMER OPERATOR'
    You must distinguish between the BUYER (Payer) and the OPERATOR (End User).
    
    **Analysis Steps:**
    1. Read the full text. Who is receiving/using the equipment?
    2. **Navy vs Foreign Assistance Rule**:
       - If the text says "Navy awarded contract...", the Operator is **Navy**.
       - "Foreign Assistance" is ONLY for cases where Country A buys gear to give to Country B (excluding Ukraine).
       - "Ukraine (Assistance)" is ONLY if Country A buys gear specifically for Ukraine.
       - "Defence Wide": Agencies like DLA, DARPA, DTRA.
       
    ### ENTITY RULES:
    1. **Customer Country**: The country paying/signing. (If Org buys for Ukraine -> Org is Customer).
    2. **Supplier Country**: Location of the performing entity (e.g. "BAE Systems Inc (USA)" -> USA).
    3. **Domestic Content**: Look for "license production" or "local assembly" cues.

    Return JSON.
    """

    user_prompt = f"""
    Analyze this text: "{paragraph}"

    Return JSON:
    {{
        "operator_reasoning": "Explain WHY you chose the operator based on the text context",
        "raw_customer_country": "Country Name or Org",
        "raw_customer_operator": "Army / Navy / Air Force / Defence Wide / Ukraine (Assistance) / Foreign Assistance / Other",
        "raw_supplier_country": "Country Name",
        "is_license_production": boolean,
        "is_local_assembly": boolean
    }}
    """

    # --- CALL LLM ---
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        raw = json.loads(completion.choices[0].message.content)
    except Exception as e:
        return {"Error": str(e)}

    # --- POST-PROCESSING ---
    cust_country = raw.get("raw_customer_country", "Unknown")
    supp_country = raw.get("raw_supplier_country", "Unknown")
    
    # Use mapped regions
    cust_region = get_region_for_country(cust_country)
    supp_region = get_region_for_country(supp_country)

    # Domestic Content Logic
    flags = {
        "is_license_production": raw.get("is_license_production", False),
        "is_local_assembly": raw.get("is_local_assembly", False)
    }
    dom_content = determine_domestic_content(cust_country, supp_country, flags)

    return {
        "Customer Region": cust_region,
        "Customer Country": cust_country,
        "Customer Operator": raw.get("raw_customer_operator", "Unknown"),
        "Operator Reasoning": raw.get("operator_reasoning", "No reasoning provided"), # Validates logic
        "Supplier Region": supp_region,
        "Supplier Country": supp_country,
        "Domestic Content": dom_content
    }

In [48]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=f'{os.environ.get("LLMFOUNDRY_TOKEN")}:my-test-project',
    base_url="https://llmfoundry.straive.com/openai/v1/",
    temperature=0
)

tools = [geography_extractor]
llm_with_tools = llm.bind_tools(tools)

def call_model(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

tool_node = ToolNode(tools)

def should_continue(state: MessagesState):
    if state["messages"][-1].tool_calls:
        return "tools"
    return END

workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

app = workflow.compile()

In [49]:
if __name__ == "__main__":
    # Test: A tricky case where "Foreign Assistance" might be wrongly triggered if not careful
    test_text = "The Johns Hopkins University Applied Physics Laboratory, Laurel, Maryland, is awarded a $4,396,000,000 cost-plus-fixed-fee, indefinite-delivery/indefinite-quantity contract for research, development, engineering, and test and evaluation for programs throughout the Department of Defense within its core competency areas including strategic systems test and evaluation; submarine security and survivability; space science and engineering; combat systems and guided missiles; air and missile defense and power projection; information technology, simulation, modeling, and operations analysis; and mission related research, development, test and evaluation. This contract includes options which, if exercised, would bring the cumulative value of this contract to $10,600,000,000. Work will be performed in Laurel, Maryland, and is expected to be completed by August 2027. If all options are exercised, work will continue through August 2032. Fiscal 2022 research, development, test and evaluation (Navy) funds in the amount of $6,803,376 will be obligated at time of award and will not expire at the end of the current fiscal year. This contract was not competitively procured in accordance with 10 U.S. Code 2304(c)(3)(B), as implemented in Defense Federal Acquisition Regulation 6.302-3 — industrial mobilization; engineering, developmental, or research capability; or expert services. The Naval Sea Systems Command, Washington, D.C., is the contracting activity (N00024-22-D-6404)."
    
    print("--- Running Geography Agent ---")
    inputs = {"messages": [HumanMessage(content=f"Extract geography info: {test_text}")]}
    
    for chunk in app.stream(inputs, stream_mode="values"):
        message = chunk["messages"][-1]
        message.pretty_print()

--- Running Geography Agent ---
================================ Human Message =================================

Extract geography info: The Johns Hopkins University Applied Physics Laboratory, Laurel, Maryland, is awarded a $4,396,000,000 cost-plus-fixed-fee, indefinite-delivery/indefinite-quantity contract for research, development, engineering, and test and evaluation for programs throughout the Department of Defense within its core competency areas including strategic systems test and evaluation; submarine security and survivability; space science and engineering; combat systems and guided missiles; air and missile defense and power projection; information technology, simulation, modeling, and operations analysis; and mission related research, development, test and evaluation. This contract includes options which, if exercised, would bring the cumulative value of this contract to $10,600,000,000. Work will be performed in Laurel, Maryland, and is expected to be completed by August

## Merger Code

In [53]:
import os
import json
import difflib
import pandas as pd
import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta
import getpass
from typing import Annotated, TypedDict, List
import re

# LangChain / LangGraph Imports
from langchain_core.messages import HumanMessage, AnyMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from openai import OpenAI

# ==============================================================================
# 1. CONFIGURATION & FILE PATHS
# ==============================================================================

# ⬇️ UPDATE PATHS HERE ⬇️
TAXONOMY_PATH = r'C:\Users\mukeshkr\Desktop\DefenseExtraction\testing\taxonomy.json'
SUPPLIERS_PATH = r'C:\Users\mukeshkr\Desktop\DefenseExtraction\testing\suppliers.json'
INPUT_EXCEL_PATH = r"C:\Users\mukeshkr\Desktop\DefenseExtraction\data\source_file.xlsx"
OUTPUT_EXCEL_PATH = "Processed_Defense_Data.xlsx"

# Setup API Key
if "LLMFOUNDRY_TOKEN" not in os.environ:
    os.environ["LLMFOUNDRY_TOKEN"] = getpass.getpass("Enter the LLM Foundry API Key: ")

# Shared Client
client = OpenAI(
    api_key=f'{os.environ.get("LLMFOUNDRY_TOKEN")}:my-test-project',
    base_url="https://llmfoundry.straive.com/openai/v1/",
)

# --- FILE LOADING HELPERS ---
def load_json_file(filename, default_value):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            print(f"✅ Loaded {filename}")
            return json.load(f)
    except Exception as e:
        print(f"⚠️ Warning: Could not load {filename} ({e}). Using default.")
        return default_value

# 1. Load Taxonomy
raw_taxonomy = load_json_file(TAXONOMY_PATH, {})
TAXONOMY_STR = json.dumps(raw_taxonomy, separators=(',', ':'))

# 2. Load Suppliers
SUPPLIER_LIST = load_json_file(SUPPLIERS_PATH, [
    "Dell Inc", "Boeing", "Lockheed Martin", "Raytheon Technologies", 
    "Northrop Grumman", "L3Harris", "BAE Systems", "General Dynamics"
])

# 3. System Rules
RULE_BOOK = {
    "defensive_countermeasures": {
        "triggers": ["flare", "chaff", "countermeasure", "decoy", "mju-", "ale-"],
        "guidance": "Market Segment: 'C4ISR Systems', System Type (General): 'Defensive Systems', Specific: 'Defensive Aid Suite'"
    },
    "radars_and_sensors": {
        "triggers": ["radar", "sonar", "sensor", "an/apy", "an/tpy"],
        "guidance": "Market Segment: 'C4ISR Systems', System Type (General): 'Sensors'"
    },
    "ammunition": {
        "triggers": ["cartridge", "round", "projectile", " 5.56", " 7.62", "ammo"],
        "guidance": "Market Segment: 'Weapon Systems', System Type (General): 'Ammunition'"
    }
}

# 4. Geography Mapping
GEOGRAPHY_MAPPING = {
    "North America": ["USA", "United States", "US", "Canada", "America"],
    "Europe": ["UK", "United Kingdom", "Ukraine", "Germany", "France", "Italy", "Spain", "Poland", "Netherlands", "Norway", "Sweden", "Finland", "Denmark", "Belgium"],
    "Asia-Pacific": ["Australia", "Japan", "South Korea", "Taiwan", "India", "Singapore", "New Zealand"],
    "Middle East and North Africa": ["Israel", "Saudi Arabia", "UAE", "Egypt", "Qatar", "Kuwait", "Iraq"],
    "International Organisations": ["NATO", "EU", "IFU", "UN", "NSPA"]
}

# ==============================================================================
# 2. HELPER FUNCTIONS
# ==============================================================================

def get_best_supplier_match(extracted_name):
    """
    Fuzzy matches the extracted name against the loaded SUPPLIER_LIST.
    """
    if not extracted_name or extracted_name.lower() in ["unknown", "n/a"]: 
        return "Unknown"
    
    clean_name = extracted_name.strip()
    
    # 1. Exact Match (Case Insensitive)
    supplier_map = {s.lower(): s for s in SUPPLIER_LIST}
    if clean_name.lower() in supplier_map: 
        return supplier_map[clean_name.lower()]
    
    # 2. Fuzzy Match (Cutoff 0.6 = 60% similarity)
    matches = difflib.get_close_matches(clean_name, SUPPLIER_LIST, n=1, cutoff=0.6)
    
    return matches[0] if matches else clean_name

def calculate_mro_months(start_date_str, end_date_text, program_type):
    """
    Calculates duration only if Program Type is MRO/Support.
    """
    if program_type != "MRO/Support":
        return "Not Applicable"

    try:
        if not start_date_str or not end_date_text or end_date_text.lower() in ["unknown", "n/a"]: 
            return "Not Applicable"
        
        start = pd.to_datetime(start_date_str, dayfirst=True)
        end = parser.parse(end_date_text, fuzzy=True)
        
        # Calculate difference
        diff = relativedelta(end, start)
        total_months = (diff.years * 12) + diff.months
        
        return str(max(0, int(total_months)))
    except: 
        return "Not Applicable"

def get_region_for_country(country_name):
    """Robust lookup handling casing/whitespace."""
    if not country_name or country_name.lower() in ["unknown", "n/a", "not applicable"]: 
        return "Unknown"
    
    clean = country_name.strip().lower()
    
    # Direct Aliases
    if clean in ["us", "usa", "united states", "united states of america"]: return "North America"
    if clean in ["uk", "britain", "great britain"]: return "Europe"
    
    # Map Search
    for region, countries in GEOGRAPHY_MAPPING.items():
        if any(c.lower() == clean for c in countries): 
            return region
    return "Unknown"

# ==============================================================================
# 3. TOOL DEFINITIONS (AGENTS)
# ==============================================================================

# --- AGENT 1: SOURCING ---
class SourcingInput(BaseModel):
    paragraph: str = Field(description="Contract text.")
    url: str = Field(description="Source URL.")
    date: str = Field(description="Contract Date.")

@tool("sourcing_extractor")
def sourcing_extractor(paragraph: str, url: str, date: str):
    """Stage 1: Prepares Metadata."""
    reported_date = datetime.datetime.now().strftime("%Y-%m-%d")
    notes = "Standard extraction."
    if "modification" in paragraph.lower(): notes = "Contract Modification."
    if "split" in paragraph.lower(): notes = "Split award detected."

    return {
        "Description of Contract": paragraph,
        "Additional Notes (Internal Only)": notes,
        "Source Link(s)": url,
        "Contract Date": date,
        "Reported Date (By SGA)": reported_date
    }

# --- AGENT 2: GEOGRAPHY ---
class GeographyInput(BaseModel):
    paragraph: str = Field(description="Contract text.")

@tool("geography_extractor")
def geography_extractor(paragraph: str):
    """Stage 2: Geography Logic."""
    sys_prompt = """
    Extract: Customer Country, Supplier Country.
    Logic: If 'Navy awarded...', Operator is Navy. 
    Return JSON: {"Customer Country": "...", "Customer Operator": "...", "Supplier Country": "..."}
    """
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini", messages=[{"role": "system", "content": sys_prompt}, {"role": "user", "content": paragraph}], temperature=0, response_format={"type": "json_object"}
        )
        raw = json.loads(completion.choices[0].message.content)
    except Exception: raw = {}

    cust = raw.get("Customer Country", "Unknown")
    supp = raw.get("Supplier Country", "Unknown")
    
    # Logic
    domestic = "Indigenous" if cust.lower() == supp.lower() else "Imported"
    if "united states" in cust.lower() and "usa" in supp.lower(): domestic = "Indigenous"

    return {
        "Customer Region": get_region_for_country(cust),
        "Customer Country": cust,
        "Customer Operator": raw.get("Customer Operator", "Unknown"),
        "Supplier Region": get_region_for_country(supp),
        "Supplier Country": supp,
        "Domestic Content": domestic
    }

# --- AGENT 3: SYSTEM (FIXED PROMPT) ---
class SystemInput(BaseModel):
    paragraph: str = Field(description="Contract text.")

@tool("system_classifier")
def system_classifier(paragraph: str):
    """Stage 3: System Logic."""
    lower_text = paragraph.lower()
    hints = [f"RULE: {v['guidance']}" for k, v in RULE_BOOK.items() if any(t in lower_text for t in v['triggers'])]
    hint_str = "\n".join(hints)

    # 🛑 CRITICAL FIX: Prompt now strictly forbids nested JSON/lists
    sys_prompt = f"""
    You are a Defense Analyst. Classify using the provided Taxonomy.
    
    REFERENCE TAXONOMY:
    {TAXONOMY_STR}
    
    SPECIAL RULES:
    {hint_str}
    
    OUTPUT INSTRUCTIONS:
    1. Return a FLAT JSON object.
    2. Values must be simple STRINGS (the category name).
    3. DO NOT return lists, definitions, or nested objects like {{'name': ...}}.
    4. If Specific Type is not found, use "Not Applicable".
    
    Example Correct Output:
    {{
      "Market Segment": "Naval Platforms",
      "System Type (General)": "Surface Combatants",
      "System Type (Specific)": "Destroyers",
      "System Name (General)": "USS Ross",
      "System Name (Specific)": "DDG 71",
      "System Piloting": "Crewed"
    }}
    """
    
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini", messages=[{"role": "system", "content": sys_prompt}, {"role": "user", "content": paragraph}], temperature=0, response_format={"type": "json_object"}
        )
        result = json.loads(completion.choices[0].message.content)
        
        # 🛑 EXTRA SAFETY: If LLM still returns a list/dict, force it to string
        for k, v in result.items():
            if isinstance(v, (list, dict)):
                result[k] = str(v) # Fallback to string representation to avoid crash
        return result
        
    except Exception: return {}

# --- AGENT 4: CONTRACT ---
class ContractInfoInput(BaseModel):
    paragraph: str = Field(description="Contract text.")
    contract_date: str = Field(description="Signed date.")

@tool("contract_extractor")
def contract_extractor(paragraph: str, contract_date: str):
    """
    Stage 4: Extracts Financials, Program Type, and Dates based on strict SOP.
    """
    
    system_instruction = """
    You are a Defense Contract Financial Analyst. Extract data strictly following these SOP rules:

    1. **Supplier Name**: Extract the exact company name text (e.g., "Dell Marketing L.P."). Do not abbreviate.
    
    2. **Program Type**:
       - **Training**: ONLY for military training *services* (instruction/courses).
         *CRITICAL EXCEPTION*: Purchase of training aircraft, simulators, or hardware is **Procurement**.
       - **Procurement**: Buying NEW products, hardware, software licenses, or construction.
       - **MRO/Support**: Maintenance, Repair, Logistics, Sustainment of EXISTING equipment.
       - **RDT&E**: Research, Prototyping, or Engineering Services.
       - **Upgrade**: Buying components to modernize/upgrade existing gear.
       - **Other Service**: Consulting, IT support, Staffing, or generic services not fitting above.
    
    3. **Value Certainty**: 
       - **Confirmed**: Specific obligated amount or fixed-price stated.
       - **Estimated**: "Potential value", "Ceiling", "Maximum value", "ID/IQ" (Indefinite Delivery), or if value is shared among multiple awardees.
    
    4. **Quantity**: 
       - Extract unit count if available (e.g., "50 missiles").
       - Default to "Not Applicable" for MRO, RDT&E, or Services unless quantifiable.

    5. **G2G/B2G**:
       - **G2G**: If "Foreign Military Sales" (FMS) is mentioned.
       - **B2G**: Default (Direct Commercial Sales).

    6. **Dates**:
       - Extract the "Completion Date" from the text for MRO calculations.

    Return JSON.
    """

    user_prompt = f"""
    Analyze this contract:
    "{paragraph}"
    
    Signed Date: {contract_date}

    Return strictly this JSON format:
    {{
        "raw_supplier_name": "Extract exact company name",
        "program_type": "Procurement/Training/MRO/Support/RDT&E/Upgrade/Other Service",
        "value_million_raw": "Extract numeric value (e.g. 10.5)",
        "currency_code": "e.g. USD$",
        "value_certainty": "Confirmed/Estimated",
        "quantity": "Number or 'Not Applicable'",
        "completion_date_text": "Extracted date string or null",
        "g2g_b2g": "G2G/B2G",
        "value_note": "Note if split/ambiguous or 'Not Applicable'"
    }}
    """
    
    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        raw = json.loads(completion.choices[0].message.content)
    except Exception as e:
        return {"Error": str(e)}

    # --- POST-PROCESSING LOGIC ---

    # 1. Supplier Matching (Fuzzy Match against JSON list)
    final_supplier = get_best_supplier_match(raw.get("raw_supplier_name"))

    # 2. Program Type Validation
    prog_type = raw.get("program_type", "Unknown")

    # 3. MRO Duration (Strictly only for MRO/Support)
    mro_months = calculate_mro_months(contract_date, raw.get("completion_date_text"), prog_type)

    # 4. Value Formatting (Millions, 3 decimals)
    try:
        val_str = str(raw.get("value_million_raw", "0")).replace(',', '').replace('$', '')
        val_float = float(val_str)
        val_formatted = "{:.3f}".format(val_float)
    except:
        val_formatted = "0.000"

    # 5. Signing Date Extraction
    try:
        dt = pd.to_datetime(contract_date)
        sign_month = dt.strftime("%B")
        sign_year = str(dt.year)
    except:
        sign_month, sign_year = "Unknown", "Unknown"
        
    # 6. Value Note / Split Logic
    val_note = raw.get("value_note", "Not Applicable")
    if "split" in paragraph.lower() and val_note == "Not Applicable":
        val_note = "Split contract; value distribution unclear."

    return {
        "Supplier Name": final_supplier,
        "Program Type": prog_type,
        "Expected MRO Contract Duration (Months)": mro_months,
        "Quantity": raw.get("quantity", "Not Applicable"),
        "Value Certainty": raw.get("value_certainty", "Confirmed"),
        "Value (Million)": val_formatted,
        "Currency": raw.get("currency_code", "USD$"),
        "Value (USD$ Million)": val_formatted, # Assuming input is USD
        "Value Note (If Any)": val_note,
        "G2G/B2G": raw.get("g2g_b2g", "B2G"),
        "Signing Month": sign_month,
        "Signing Year": sign_year
    }
# ==============================================================================
# 4. LANGGRAPH PIPELINE
# ==============================================================================

class AgentState(TypedDict):
    input_text: str
    input_date: str
    input_url: str
    final_data: dict 
    messages: Annotated[List[AnyMessage], add_messages]

def stage_1_sourcing(state: AgentState):
    res = sourcing_extractor.invoke({
        "paragraph": state["input_text"], "url": state["input_url"], "date": state["input_date"]
    })
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}

def stage_2_geography(state: AgentState):
    res = geography_extractor.invoke({"paragraph": state["input_text"]})
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}

def stage_3_system(state: AgentState):
    res = system_classifier.invoke({"paragraph": state["input_text"]})
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}

def stage_4_contract(state: AgentState):
    res = contract_extractor.invoke({
        "paragraph": state["input_text"], "contract_date": state["input_date"]
    })
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}

workflow = StateGraph(AgentState)
workflow.add_node("Stage1", stage_1_sourcing)
workflow.add_node("Stage2", stage_2_geography)
workflow.add_node("Stage3", stage_3_system)
workflow.add_node("Stage4", stage_4_contract)

workflow.add_edge(START, "Stage1")
workflow.add_edge("Stage1", "Stage2")
workflow.add_edge("Stage2", "Stage3")
workflow.add_edge("Stage3", "Stage4")
workflow.add_edge("Stage4", END)

app = workflow.compile()

# ==============================================================================
# 5. EXECUTION & FORMATTING
# ==============================================================================

if __name__ == "__main__":
    
    print(f"📂 Loading Input File: {INPUT_EXCEL_PATH}...")
    
    try:
        # 1. Read Excel
        df_input = pd.read_excel(INPUT_EXCEL_PATH)
        
        # Verify Columns
        required_cols = ["Source URL", "Contract Date", "Contract Description"]
        if not all(col in df_input.columns for col in required_cols):
            raise ValueError(f"Excel file must contain columns: {required_cols}")

        print(f"🚀 Processing {len(df_input)} rows...")
        results = []

        # 2. Iterate and Process
        for index, row in df_input.iterrows():
            print(f"   -> Row {index + 1}...")
            
            # Handle inputs
            desc = str(row['Contract Description']) if pd.notna(row['Contract Description']) else ""
            c_date = str(row['Contract Date']) if pd.notna(row['Contract Date']) else str(datetime.date.today())
            c_url = str(row['Source URL']) if pd.notna(row['Source URL']) else ""

            initial_state = {
                "input_text": desc,
                "input_date": c_date,
                "input_url": c_url,
                "final_data": {},
                "messages": []
            }
            
            output_state = app.invoke(initial_state)
            results.append(output_state["final_data"])

        # 3. FORMATTING (Strict Column Order)
        df_final = pd.DataFrame(results)

        # The Exact Column Order you requested
        FINAL_COLUMNS = [
            "Customer Region", "Customer Country", "Customer Operator",
            "Supplier Region", "Supplier Country", "Domestic Content",
            "Market Segment", "System Type (General)", "System Type (Specific)",
            "System Name (General)", "System Name (Specific)", "System Piloting",
            "Supplier Name", "Program Type", "Expected MRO Contract Duration (Months)",
            "Quantity", "Value Certainty", "Value (Million)", "Currency",
            "Value (USD$ Million)", "Value Note (If Any)", "G2G/B2G",
            "Signing Month", "Signing Year", "Description of Contract",
            "Additional Notes (Internal Only)", "Source Link(s)",
            "Contract Date", "Reported Date (By SGA)"
        ]
        
        # Reorder and Fill Missing Columns with empty strings
        df_final = df_final.reindex(columns=FINAL_COLUMNS, fill_value="")

        # 4. Save
        df_final.to_excel(OUTPUT_EXCEL_PATH, index=False)
        
        print("\n✅ Processing Complete!")
        print(f"💾 File saved to: {OUTPUT_EXCEL_PATH}")
        print(df_final.head().to_string())

    except Exception as e:
        print(f"\n❌ Error: {e}")

✅ Loaded C:\Users\mukeshkr\Desktop\DefenseExtraction\testing\taxonomy.json
✅ Loaded C:\Users\mukeshkr\Desktop\DefenseExtraction\testing\suppliers.json
📂 Loading Input File: C:\Users\mukeshkr\Desktop\DefenseExtraction\data\source_file.xlsx...
🚀 Processing 1 rows...
   -> Row 1...

✅ Processing Complete!
💾 File saved to: Processed_Defense_Data.xlsx
  Customer Region Customer Country Customer Operator Supplier Region Supplier Country Domestic Content   Market Segment System Type (General) System Type (Specific) System Name (General) System Name (Specific) System Piloting                      Supplier Name Program Type Expected MRO Contract Duration (Months)        Quantity Value Certainty Value (Million) Currency Value (USD$ Million)                                 Value Note (If Any) G2G/B2G Signing Month Signing Year                                                                                                                                                                             

C:\Users\mukeshkr\AppData\Local\Temp\ipykernel_11572\599177818.py:120: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  start = pd.to_datetime(start_date_str, dayfirst=True)
